In [3]:
import librosa 
import numpy as np
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [4]:
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = '../dataset/audio/'

#metadata = pd.read_csv('../dataset/dataset.csv')
metadata = pd.read_csv('../dataset/datasetM2.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class_name"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files') 

Finished feature extraction from  207  files


In [5]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

Using TensorFlow backend.


In [6]:
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [7]:
### store the preprocessed data for use in the next notebook

%store x_train 
%store x_test 
%store y_train 
%store y_test 
%store yy 
%store le

Stored 'x_train' (ndarray)
Stored 'x_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)
Stored 'yy' (ndarray)
Stored 'le' (LabelEncoder)


In [8]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [10]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [11]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

In [12]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 350
num_batch_size = 40

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 165 samples, validate on 42 samples
Epoch 1/350
165/165 [==============================] - 0s 1ms/step - loss: 69.9489 - accuracy: 0.2970 - val_loss: 41.7552 - val_accuracy: 0.4524

Epoch 00001: val_loss improved from inf to 41.75522, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/350
165/165 [==============================] - 0s 121us/step - loss: 52.3332 - accuracy: 0.3515 - val_loss: 18.9408 - val_accuracy: 0.4524

Epoch 00002: val_loss improved from 41.75522 to 18.94076, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 3/350
165/165 [==============================] - 0s 121us/step - loss: 39.1402 - accuracy: 0.3394 - val_loss: 17.7359 - val_accuracy: 0.4524

Epoch 00003: val_loss improved from 18.94076 to 17.73587, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/350
165/165 [==============================] - 0s 145us/step - loss: 35.9513 - accuracy: 0.4061 - val_loss: 5.5204 - val_accuracy: 0.4524

Epoch 00004: val_loss impr

165/165 [==============================] - 0s 170us/step - loss: 2.8750 - accuracy: 0.4485 - val_loss: 1.1051 - val_accuracy: 0.4048

Epoch 00037: val_loss did not improve from 0.99868
Epoch 38/350
165/165 [==============================] - 0s 170us/step - loss: 2.8420 - accuracy: 0.4545 - val_loss: 1.1180 - val_accuracy: 0.4048

Epoch 00038: val_loss did not improve from 0.99868
Epoch 39/350
165/165 [==============================] - 0s 170us/step - loss: 2.9233 - accuracy: 0.4182 - val_loss: 1.1424 - val_accuracy: 0.4048

Epoch 00039: val_loss did not improve from 0.99868
Epoch 40/350
165/165 [==============================] - 0s 194us/step - loss: 2.3747 - accuracy: 0.4485 - val_loss: 1.1511 - val_accuracy: 0.4048

Epoch 00040: val_loss did not improve from 0.99868
Epoch 41/350
165/165 [==============================] - 0s 170us/step - loss: 2.3674 - accuracy: 0.4667 - val_loss: 1.1492 - val_accuracy: 0.4048

Epoch 00041: val_loss did not improve from 0.99868
Epoch 42/350
165/165 [=

165/165 [==============================] - 0s 170us/step - loss: 1.3713 - accuracy: 0.4788 - val_loss: 1.0652 - val_accuracy: 0.4048

Epoch 00078: val_loss did not improve from 0.99868
Epoch 79/350
165/165 [==============================] - 0s 170us/step - loss: 1.3162 - accuracy: 0.4667 - val_loss: 1.0645 - val_accuracy: 0.4048

Epoch 00079: val_loss did not improve from 0.99868
Epoch 80/350
165/165 [==============================] - 0s 170us/step - loss: 1.3838 - accuracy: 0.4788 - val_loss: 1.0625 - val_accuracy: 0.4048

Epoch 00080: val_loss did not improve from 0.99868
Epoch 81/350
165/165 [==============================] - 0s 194us/step - loss: 1.3065 - accuracy: 0.4485 - val_loss: 1.0576 - val_accuracy: 0.4048

Epoch 00081: val_loss did not improve from 0.99868
Epoch 82/350
165/165 [==============================] - 0s 145us/step - loss: 1.2825 - accuracy: 0.5030 - val_loss: 1.0534 - val_accuracy: 0.4048

Epoch 00082: val_loss did not improve from 0.99868
Epoch 83/350
165/165 [=

165/165 [==============================] - 0s 170us/step - loss: 1.3273 - accuracy: 0.4727 - val_loss: 1.0482 - val_accuracy: 0.4048

Epoch 00119: val_loss did not improve from 0.99868
Epoch 120/350
165/165 [==============================] - 0s 170us/step - loss: 1.1567 - accuracy: 0.5212 - val_loss: 1.0484 - val_accuracy: 0.4048

Epoch 00120: val_loss did not improve from 0.99868
Epoch 121/350
165/165 [==============================] - 0s 194us/step - loss: 1.1997 - accuracy: 0.5152 - val_loss: 1.0487 - val_accuracy: 0.4048

Epoch 00121: val_loss did not improve from 0.99868
Epoch 122/350
165/165 [==============================] - 0s 145us/step - loss: 1.2160 - accuracy: 0.4727 - val_loss: 1.0487 - val_accuracy: 0.4048

Epoch 00122: val_loss did not improve from 0.99868
Epoch 123/350
165/165 [==============================] - 0s 194us/step - loss: 1.1492 - accuracy: 0.5273 - val_loss: 1.0487 - val_accuracy: 0.4048

Epoch 00123: val_loss did not improve from 0.99868
Epoch 124/350
165/1

165/165 [==============================] - 0s 170us/step - loss: 1.1755 - accuracy: 0.5152 - val_loss: 1.0546 - val_accuracy: 0.4048

Epoch 00160: val_loss did not improve from 0.99868
Epoch 161/350
165/165 [==============================] - 0s 194us/step - loss: 1.2367 - accuracy: 0.5333 - val_loss: 1.0546 - val_accuracy: 0.4048

Epoch 00161: val_loss did not improve from 0.99868
Epoch 162/350
165/165 [==============================] - 0s 194us/step - loss: 1.1254 - accuracy: 0.4970 - val_loss: 1.0545 - val_accuracy: 0.4048

Epoch 00162: val_loss did not improve from 0.99868
Epoch 163/350
165/165 [==============================] - 0s 194us/step - loss: 1.1788 - accuracy: 0.4788 - val_loss: 1.0544 - val_accuracy: 0.4048

Epoch 00163: val_loss did not improve from 0.99868
Epoch 164/350
165/165 [==============================] - 0s 170us/step - loss: 1.1105 - accuracy: 0.4848 - val_loss: 1.0544 - val_accuracy: 0.4048

Epoch 00164: val_loss did not improve from 0.99868
Epoch 165/350
165/1

165/165 [==============================] - 0s 170us/step - loss: 1.0695 - accuracy: 0.5091 - val_loss: 1.0550 - val_accuracy: 0.4286

Epoch 00201: val_loss did not improve from 0.99868
Epoch 202/350
165/165 [==============================] - 0s 170us/step - loss: 1.1155 - accuracy: 0.5333 - val_loss: 1.0506 - val_accuracy: 0.4762

Epoch 00202: val_loss did not improve from 0.99868
Epoch 203/350
165/165 [==============================] - 0s 145us/step - loss: 1.0665 - accuracy: 0.5152 - val_loss: 1.0480 - val_accuracy: 0.4762

Epoch 00203: val_loss did not improve from 0.99868
Epoch 204/350
165/165 [==============================] - 0s 170us/step - loss: 1.1636 - accuracy: 0.4788 - val_loss: 1.0454 - val_accuracy: 0.4762

Epoch 00204: val_loss did not improve from 0.99868
Epoch 205/350
165/165 [==============================] - 0s 242us/step - loss: 1.0671 - accuracy: 0.4970 - val_loss: 1.0434 - val_accuracy: 0.4762

Epoch 00205: val_loss did not improve from 0.99868
Epoch 206/350
165/1

165/165 [==============================] - 0s 169us/step - loss: 1.0420 - accuracy: 0.5576 - val_loss: 1.0417 - val_accuracy: 0.4762

Epoch 00242: val_loss did not improve from 0.99868
Epoch 243/350
165/165 [==============================] - 0s 145us/step - loss: 1.1569 - accuracy: 0.4727 - val_loss: 1.0418 - val_accuracy: 0.4762

Epoch 00243: val_loss did not improve from 0.99868
Epoch 244/350
165/165 [==============================] - 0s 169us/step - loss: 1.0965 - accuracy: 0.4788 - val_loss: 1.0397 - val_accuracy: 0.4762

Epoch 00244: val_loss did not improve from 0.99868
Epoch 245/350
165/165 [==============================] - 0s 194us/step - loss: 1.1277 - accuracy: 0.4848 - val_loss: 1.0282 - val_accuracy: 0.4762

Epoch 00245: val_loss did not improve from 0.99868
Epoch 246/350
165/165 [==============================] - 0s 170us/step - loss: 1.0976 - accuracy: 0.5152 - val_loss: 1.0231 - val_accuracy: 0.4762

Epoch 00246: val_loss did not improve from 0.99868
Epoch 247/350
165/1

165/165 [==============================] - 0s 170us/step - loss: 1.1021 - accuracy: 0.5515 - val_loss: 0.9807 - val_accuracy: 0.4762

Epoch 00281: val_loss did not improve from 0.97942
Epoch 282/350
165/165 [==============================] - 0s 169us/step - loss: 1.0838 - accuracy: 0.5030 - val_loss: 0.9769 - val_accuracy: 0.4762

Epoch 00282: val_loss improved from 0.97942 to 0.97691, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 283/350
165/165 [==============================] - 0s 111us/step - loss: 1.0541 - accuracy: 0.5394 - val_loss: 0.9742 - val_accuracy: 0.4762

Epoch 00283: val_loss improved from 0.97691 to 0.97416, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 284/350
165/165 [==============================] - 0s 139us/step - loss: 1.0569 - accuracy: 0.5394 - val_loss: 0.9732 - val_accuracy: 0.4762

Epoch 00284: val_loss improved from 0.97416 to 0.97322, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 285/350
165/165 [=========

Epoch 317/350
165/165 [==============================] - 0s 194us/step - loss: 1.0287 - accuracy: 0.5273 - val_loss: 0.9372 - val_accuracy: 0.5000

Epoch 00317: val_loss improved from 0.94076 to 0.93720, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 318/350
165/165 [==============================] - 0s 196us/step - loss: 1.0300 - accuracy: 0.5515 - val_loss: 0.9361 - val_accuracy: 0.5000

Epoch 00318: val_loss improved from 0.93720 to 0.93605, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 319/350
165/165 [==============================] - 0s 177us/step - loss: 1.0320 - accuracy: 0.5394 - val_loss: 0.9387 - val_accuracy: 0.5000

Epoch 00319: val_loss did not improve from 0.93605
Epoch 320/350
165/165 [==============================] - 0s 218us/step - loss: 1.0209 - accuracy: 0.5697 - val_loss: 0.9513 - val_accuracy: 0.4762

Epoch 00320: val_loss did not improve from 0.93605
Epoch 321/350
165/165 [==============================] - 0s 274us/step - loss: 1

In [13]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.6484848260879517
Testing Accuracy:  0.4761904776096344


In [14]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])


In [15]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [18]:
# test

filename = '../dataset/audio/test_wp.wav' 
print_prediction(filename) 

The predicted class is: tap 

double_tap 		 :  0.28137969970703125000000000000000
long_tap 		 :  0.05158194899559020996093750000000
swipe 		 :  0.16705252230167388916015625000000
tap 		 :  0.46794164180755615234375000000000
zoom 		 :  0.03204415366053581237792968750000
